In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#* import cleaned text

df_train = pd.read_csv("/kaggle/input/nlp-disaster-tweets/train_cleaned.csv")
df_test = pd.read_csv("/kaggle/input/nlp-disaster-tweets/test_cleaned.csv")


In [ ]:
df_train

In [ ]:
df_test

In [ ]:
from transformers import AdamW, RobertaTokenizer, RobertaForSequenceClassification
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import torch

# Dataset preparation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_train["text_cleaned"], df_train["target"], test_size=0.2, random_state=42
)

# Load the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

#tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
#model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
print(f"Using device: {device}")

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)

In [ ]:
import torch

class DisasterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None, device="cuda:0"):
        self.encodings = encodings
        self.labels = labels
        self.device = device

    def __len__(self):
        return len(self.encodings['input_ids'])  


    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:  # Add labels only if they exist
            item['labels'] = torch.tensor(self.labels[idx]).to(self.device)
        return item

In [ ]:
train_dataset = DisasterDataset(train_encodings, train_labels.tolist(), device= device)
val_dataset = DisasterDataset(val_encodings, val_labels.tolist(), device = device)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# Optimizer
# optimizer = AdamW(model.parameters(), lr=5e-5)
optimizer = AdamW(
    model.parameters(),           # Model parameters to optimize
    lr=1e-5,                      # Learning rate (commonly 2e-5 to 5e-5 for BERT/RoBERTa)
    betas=(0.9, 0.999),           # Beta values for Adam; defaults are typically fine
    eps=1e-8,                     # Epsilon for numerical stability
    weight_decay=0.01,            # Weight decay to prevent overfitting
    correct_bias=True             # Whether to correct bias (default for AdamW)
)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

# Training loop
model.train()
epoch_num =50 
for epoch in range(epoch_num):  # Number of epochs
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # Move to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step(loss)

        # Update progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Predictions
model.eval()
all_preds = []
all_labels = []
for batch in val_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=-1)

    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(labels.cpu().numpy())

# F1 Score
f1 = f1_score(all_labels, all_preds)
print(f"F1 Score: {f1}")




In [ ]:
# Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
test_sentences = list(df_test['text'].values)
#test_encodings = tokenizer(test_sentences, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(df_test['text']), truncation=True, padding=True, max_length=128)

#test_inputs = torch.tensor(test_encodings['input_ids'])
#test_masks = torch.tensor(test_encodings['attention_mask'])

#test_data = DisasterDataset(test_inputs, test_masks, torch.zeros(len(test_inputs)))  # No labels in test set
#test_loader = DataLoader(test_data, batch_size=16)

test_data = DisasterDataset(test_encodings)
test_loader = DataLoader(test_data, batch_size=16)

model.eval()
predictions = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask, labels = labels)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())

submission = pd.DataFrame({'id': df_test['id'], 'target': predictions})
submission.to_csv('submission.csv', index=False)
